# Experimenting with AutoML
Explore different AutoML libraries for Scikit-Learn to see whether this can improve our predictions.

### Table of contents
* [Data loading, performance metric & preprocessing](#loaddata)
* [AutoML libraries](#automl)
    * [TPOT](#tpot)
    * [Auto-Sklearn](#autosklearn)
    * [Hyperopt-Sklearn](#hyperopt)
* [Conclusion](#conclusion)

In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, train_test_split
from tpot import TPOTRegressor
from hpsklearn import HyperoptEstimator, any_regressor, any_preprocessing
from hyperopt import tpe

train_data_file = "../data/train.csv"
test_data_file = "../data/test.csv"
tpot_predictions_file = "../results/02_predictions_tpot.csv"
hyperopt_predictions_file = "../results/02_predictions_hyperopt.csv"
SEED = 0
CV = 5

## Data loading, performance metric & preprocessing <a class="anchor"  id="loaddata"></a>
The code for loading the data, defining the performance metric and a basic preprocessing is mainly copied from [01_basic.ipynb](jupyter_notebooks/01_basic.ipynb). We also define a helper function to make and save the predictions on the test set.

Since the AutoML libraries seem to not work correctly with the `TransformedTargetRegressor`, the target is manually log transformed during data loading. The inverse transformation is performed in the prediction making helper function. For the performance metric, we use the RMSE (instead of RMSLE) since the target is already logarithmic. Note that this does not exactly yield the RMSLE (defined in [01_basic.ipynb](jupyter_notebooks/01_basic.ipynb)) which uses $\log_e (1 + y)$ instead of $\log_e (y)$. But it is close enough.

In [21]:
# load data
train_df = pd.read_csv(train_data_file)
train_df.set_index("Id", inplace=True)
target_col = "SalePrice"
log_y_train = np.log(train_df[target_col])
X_train = train_df.drop(columns=[target_col])
cat_cols = X_train.select_dtypes(include=["object"]).columns
num_cols = X_train.select_dtypes(exclude=["object"]).columns

X_test = pd.read_csv(test_data_file)
X_test.set_index("Id", inplace=True)

In [18]:
# define performance metric
neg_RMSE_scorer = make_scorer(
    mean_squared_error, greater_is_better=False, squared=False
)


def measure_performance(estimator, X, log_y, scorer=neg_RMSE_scorer, cv=CV):
    """Calculate negative RMSLE for train and validation set via cross validation.
    Note that the logarithm of the target has to be used for `log_y`."""
    cv_results = cross_validate(
        estimator=estimator,
        X=X,
        y=log_y,
        cv=cv,
        scoring=scorer,
        return_train_score=True,
    )
    train_error = cv_results["train_score"].mean()
    validation_error = cv_results["test_score"].mean()
    return train_error, validation_error

In [5]:
# basic preprocessing
simple_imputer = ColumnTransformer(
    transformers=[
        (
            "num_imputer",
            SimpleImputer(strategy="mean", keep_empty_features=True),
            num_cols,
        ),
        (
            "cat_imputer",
            SimpleImputer(strategy="most_frequent", keep_empty_features=True),
            cat_cols,
        ),
    ],
    verbose_feature_names_out=False,
)
simple_imputer.set_output(transform="pandas")

ordinal_encoder = ColumnTransformer(
    transformers=[
        (
            "ordinal_encoder",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
            cat_cols,
        )
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)
ordinal_encoder.set_output(transform="pandas")


def build_pipeline(estimator, imputer=simple_imputer, encoder=ordinal_encoder):
    preprocessing = Pipeline(steps=[("imputer", imputer), ("encoder", encoder)])
    pipeline = Pipeline(
        steps=[
            ("preprocessing", preprocessing),
            ("estimator", estimator),
        ]
    )
    return pipeline

In [22]:
# helper function for making test predictions:
def safe_log_inverse(log_y):
    # clip values in order to avoid overflows in exponential function
    log_y_clipped = np.clip(log_y, -sys.float_info.max, np.log(sys.float_info.max))
    y = np.exp(log_y_clipped)
    return y


def make_test_predictions(pipe, file, X_train, log_y_train, X_test):
    """Create test predictions and save them."""
    pipe.fit(X_train, log_y_train)
    log_pred_test = pipe.predict(X=X_test)
    pred_test = safe_log_inverse(log_y=log_pred_test)
    prediction_df = pd.DataFrame({"Id": X_test.index, "SalePrice": pred_test})
    prediction_df.to_csv(file, index=False)
    return prediction_df

## AutoML libraries <a class="anchor"  id="automl"></a>
AutoML stands for Automated Machine Learning. It is a tool to automatically obtain a machine learning pipeline with a relatively good performance. This is achieved via an optimization of the model selection, hyperparameter tuning and some data preprocessing. There are different techniques for the optimzation like grid or random search, Bayesian optimization or evolutionary algorithms.


We will test and compare multiple AutoML libraries for scikit-learn: TPOT, Auto-Sklearn and HyperOpt-Sklearn.

### TPOT <a class="anchor"  id="tpot"></a>
TPOT stands for tree based pipeline optimization tool. It uses genetic programming (evolutionary algotihm) for the optimization.

The input data must be numerical only. Therefore before running TPOT, missing values are handled and categorical features are encoded.

The best found estimator is a stacking of `RidgeCV()` and `XGBRegressor` with the following parameters:
- learning_rate=0.1
- max_depth=5
- min_child_weight=6
- n_estimators=100
- objective=reg:squarederror
- subsample=0.6500000000000001.

The RMSLE of the best pipeline is roughly 0.125 on the validation set. The run time has been roughly 11min on my machine.

In [7]:
# silence warning about missing feature names
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, message="X does not have valid feature names.*"
)

In [37]:
# run TPOT:
tpot_regr = TPOTRegressor(
    generations=10,
    population_size=50,
    cv=CV,
    scoring=neg_RMSE_scorer,
    early_stop=3,
    verbosity=2,
    random_state=SEED,
    n_jobs=-1,
)
tpot_pipe = build_pipeline(estimator=tpot_regr)
tpot_pipe.fit(X_train, log_y_train)

                                                                              
Generation 1 - Current best internal CV score: -0.13139412689634072
                                                                              
Generation 2 - Current best internal CV score: -0.13095566298949715
                                                                              
Generation 3 - Current best internal CV score: -0.12884397185509527
                                                                              
Generation 4 - Current best internal CV score: -0.12859550606896564
                                                                              
Generation 5 - Current best internal CV score: -0.12859550606896564
                                                                              
Generation 6 - Current best internal CV score: -0.12859550606896564
                                                                              
Generation 7 - Current best internal CV

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(transformers=[('num_imputer',
                                                                   SimpleImputer(keep_empty_features=True),
                                                                   Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'G...
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))],
                                                    verbose_feature_names_out=False))])),
                ('estimator',
                 TPOTRegressor(early_stop=3, generations=10, n_jobs=-1,
                               population_size=50, random_state=0,
                               scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False),
                               verbosity=2))])

In [38]:
# get pipeline with best TPOT model:
best_tpot_estimator = tpot_pipe.named_steps["estimator"].fitted_pipeline_
best_tpot_pipe = build_pipeline(estimator=best_tpot_estimator)
best_tpot_pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(transformers=[('num_imputer',
                                                                   SimpleImputer(keep_empty_features=True),
                                                                   Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'G...
                                               gpu_id=None, grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.1, max_bin=None,
                                               max_cat_threshold=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None, max_depth=5,
                                               max_leaves=None,
                                               min_child_weight=6, missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=100, n_jobs=1,
                                               num_parallel_tree=None,
                                               predictor=None, random_state=0, ...))]))])

In [39]:
# measure performance (double-check):
train_error, test_error = measure_performance(
    estimator=best_tpot_pipe, X=X_train, log_y=log_y_train
)
print(f"Train error: {train_error}; Validation error: {test_error}")

Train error: -0.07341580322576022; Validation error: -0.12515826479797265


In [40]:
# make test predictions:
# make test predictions
make_test_predictions(
    pipe=best_tpot_pipe,
    file=tpot_predictions_file,
    X_train=X_train,
    log_y_train=log_y_train,
    X_test=X_test,
).head()

,Id,SalePrice
0,1461,119009.011759
1,1462,152536.866076
2,1463,170283.243485
3,1464,194041.995718
4,1465,177704.319183


### Auto-Sklearn <a class="anchor"  id="autosklearn"></a>

### Hyperopt-Sklearn <a class="anchor"  id="hyperopt"></a>
Hyperopt-Sklearn is a wrapper for HyperOpt which employs Bayesian optimization. It is especially designed for large scales and facilitates parallel computing.

The input data must be numerical only. Therefore before running Hyperopt-Sklearn, missing values are handled and categorical features are encoded.

Note that Hyperopt-Sklearn optimizes the MSE instead of RMSE. This is due to the fact that a custom wrapper around `mean_squared_error` with `squared=False` did not work as `loss_fn` in `HyperoptEstimator` since an `AllTrialsFailed` error is thrown.

The seed parameter for the `HyperoptEstimator` seems to be not working since multiple runs yield different results.

The best found estimator is a pipeline with `StandardScaler(with_std=False)` and `GradientBoostingRegressor` with the following parameters:
- alpha=0.866371138307043
- criterion='squared_error'
- learning_rate=0.10712565734276921
- max_features=0.7009553580153713
- max_leaf_nodes=10,
- n_estimators=276
- random_state=1

The RMSLE of the best pipeline is roughly 0.123 on the validation set. The run time has been roughly 1min on my machine.

Other runs of Hyperopt-Sklearn with the same parameters have yield worse outcomes.

In [86]:
# run hyperopt-sklearn
hyper_regr = HyperoptEstimator(
    regressor=any_regressor("reg"),
    preprocessing=any_preprocessing("pre"),
    algo=tpe.suggest,
    loss_fn=mean_squared_error,  # MSE
    max_evals=10,
    trial_timeout=5 * 60,
    seed=SEED,
    n_jobs=-1,
)
hyper_pipe = build_pipeline(estimator=hyper_regr)
hyper_pipe.fit(X_train, log_y_train)

100%|██████████| 10/10 [00:01<00:00,  1.77s/trial, best loss: 0.016043031588240088]


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(transformers=[('num_imputer',
                                                                   SimpleImputer(keep_empty_features=True),
                                                                   Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'G...
                 hyperopt_estimator(algo=<function suggest at 0x000002BF129884C0>,
                                    ex_preprocs=[],
                                    loss_fn=<function mean_squared_error at 0x000002BF09A88040>,
                                    n_jobs=-1,
                                    preprocessing=<hyperopt.pyll.base.Apply object at 0x000002BF1A2C8FD0>,
                                    regressor=<hyperopt.pyll.base.Apply object at 0x000002BF225F6FE0>,
                                    seed=0,
                                    space=<hyperopt.pyll.base.Apply object at 0x000002BF229C0B50>,
                                    trial_timeout=300))])

In [87]:
# get pipeline with best Hyperopt-Sklearn model:
def get_pipeline_from_dict(estimator_dict, step_names=["preprocs", "learner"]):
    from sklearn.pipeline import make_pipeline

    all_steps = []
    for step_name in step_names:
        steps = estimator_dict[step_name]
        if not isinstance(steps, tuple):
            steps = [steps]
        all_steps.append((step_name, make_pipeline(*steps)))
    pipe = Pipeline(steps=all_steps)
    return pipe


best_hyper_estimator_dict = hyper_pipe.named_steps["estimator"].best_model()
best_hyper_estimator = get_pipeline_from_dict(estimator_dict=best_hyper_estimator_dict)

best_hyper_pipe = build_pipeline(estimator=best_hyper_estimator)
best_hyper_pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(transformers=[('num_imputer',
                                                                   SimpleImputer(keep_empty_features=True),
                                                                   Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'G...
                 Pipeline(steps=[('preprocs',
                                  Pipeline(steps=[('standardscaler',
                                                   StandardScaler(with_std=False))])),
                                 ('learner',
                                  Pipeline(steps=[('gradientboostingregressor',
                                                   GradientBoostingRegressor(alpha=0.866371138307043,
                                                                             criterion='squared_error',
                                                                             learning_rate=0.10712565734276921,
                                                                             max_features=0.7009553580153713,
                                                                             max_leaf_nodes=10,
                                                                             n_estimators=276,
                                                                             random_state=1,
                                                                             verbose=False))]))]))])

In [91]:
# measure performance:
train_error, test_error = measure_performance(best_hyper_pipe, X_train, log_y_train)
print(f"Train error: {train_error}; Validation error: {test_error}")

Train error: -0.04963309009622466; Validation error: -0.12258558934610764


In [89]:
# make test predictions
make_test_predictions(
    pipe=best_hyper_pipe,
    file=hyperopt_predictions_file,
    X_train=X_train,
    log_y_train=log_y_train,
    X_test=X_test,
).head()

,Id,SalePrice
0,1461,121529.647048
1,1462,150973.163738
2,1463,179076.624520
3,1464,186709.022531
4,1465,181610.553065


## Conclusion <a class="anchor"  id="conclusion"></a>